In [1]:
import numpy as np
import tensorflow as tf
import cv2

In [5]:
class DetectorAPI:
    def __init__(self, path_to_ckpt):
        self.path_to_ckpt = path_to_ckpt
 
        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(self.path_to_ckpt, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')
 
        self.default_graph = self.detection_graph.as_default()
        self.sess = tf.Session(graph=self.detection_graph)
 
        # Definir los tensores de entrada y salida para detection_graph
        self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
        # Cada caja representa una parte de la imagen donde un objeto en particular fue detectado
        self.detection_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
        # Cada puntaje representa un nivel de precisión
        
        self.detection_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
        self.detection_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
        self.num_detections = self.detection_graph.get_tensor_by_name('num_detections:0')
 
    def processFrame(self, image):
        # El modelo esperado espera imágenes con el siguiente formato [1, None, None, 3]
        image_np_expanded = np.expand_dims(image, axis=0)
        # Comienza la detección
        
        (boxes, scores, classes, num) = self.sess.run(
            [self.detection_boxes, self.detection_scores, self.detection_classes, self.num_detections],
            feed_dict={self.image_tensor: image_np_expanded})
  
 

 
        im_height, im_width,_ = image.shape
        boxes_list = [None for i in range(boxes.shape[1])]
        #Comienza a dibujar las cajas en el lugar donde se encontró algo
        for i in range(boxes.shape[1]):
            boxes_list[i] = (int(boxes[0,i,0] * im_height),
                        int(boxes[0,i,1]*im_width),
                        int(boxes[0,i,2] * im_height),
                        int(boxes[0,i,3]*im_width))
        #Regresa las cajas que encontró
        return boxes_list, scores[0].tolist(), [int(x) for x in classes[0].tolist()], int(num[0])
 
    def close(self):
        self.sess.close()
        self.default_graph.close()
 


In [6]:
#Cargamos el path del modelo
model_path = '../../xmls/faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb'
odapi = DetectorAPI(path_to_ckpt=model_path)
threshold = 0.7
img = cv2.imread('../../Imagenes/silueta.jpg')

boxes, scores, classes, num = odapi.processFrame(img)
 
#Cuerpo Completo
for i in range(len(boxes)):
    # La clase 1, representa a los humanos
    if classes[i] == 1 and scores[i] > threshold:
        box = boxes[i]
# Dibujamos los rectángulos en las regiones obtenidas
        cv2.rectangle(img,(box[1],box[0]),(box[3],box[2]),(255,0,0),2)
 
cv2.imshow("Faster R-CNN", img)
cv2.waitKey(0)


-1